### Q1. Running Mage
version: v0.9.72

In [ ]:
!git clone https://github.com/mage-ai/rag-project


In [ ]:
!pip install python-docx elasticsearch

In [3]:
!docker pull mageai/mageai:llm

llm: Pulling from mageai/mageai

5d672725: Pulling fs layer 
3c12a9c9: Pulling fs layer 
43a5fa82: Pulling fs layer 
1019d793: Pulling fs layer 
72e87958: Pulling fs layer 
7c53cd82: Pulling fs layer 
4ae0623d: Pulling fs layer 
92e7b73b: Pulling fs layer 
1a75568b: Pulling fs layer 
fadde973: Pulling fs layer 
e10f7a13: Pulling fs layer 
aa005a6d: Pulling fs layer 
1b4caa66: Pulling fs layer 
704b6e4e: Pulling fs layer 
fbaa3ccb: Pulling fs layer 
9c2cb1f1: PullOptions complete 12B3kBBExtracting  207.8MB/211.2MBDigest: sha256:d3c4dee9f04ee466a200b1390530e03049e74f5f86541cc2ff15b4d586d866ea
Status: Downloaded newer image for mageai/mageai:llm
docker.io/mageai/mageai:llm


In [4]:
!./scripts/start.sh



/bin/bash: ./scripts/start.sh: No such file or directory


/bin/bash: mage: command not found


Q2. Reading the documents

Q3. Chunking

### Q4. Export  
document_id:6fc3236a 

Q5. Testing the retrieval


In [7]:
from elasticsearch import Elasticsearch
from sentence_transformers import SentenceTransformer
import pandas as pd


In [18]:
es_client = Elasticsearch('http://localhost:9200')
es_client.info()


ObjectApiResponse({'name': 'b691eaa5164a', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'mAY8uIQQQRm0wFjMACvY1g', 'version': {'number': '8.5.0', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': 'c94b4700cda13820dad5aa74fae6db185ca5c304', 'build_date': '2022-10-24T16:54:16.433628434Z', 'build_snapshot': False, 'lucene_version': '9.4.1', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [9]:
model_name = 'multi-qa-distilbert-cos-v1'
embedding_model = SentenceTransformer(model_name)
user_question = "When is the next cohort?"


/media/rdi/043764CF611486F91/NLP/llm-zoomcamp/llm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [19]:
v = embedding_model.encode(user_question)
index_name = 'documents_20240821_2955'


In [20]:
search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": user_question,
                    "fields": ["text", "section", "question", "course", "document_id"],
                    "type": "best_fields"
                }
            }
        }
    }
}

In [21]:
response = es_client.search(index=index_name, body=search_query)
df = pd.DataFrame(response['hits']['hits'])
df

,_index,_id,_score,_source,_ignored
0,documents_20240821_2955,YaBXcpEBGUgXSWImHL3T,8.443945,"{'text': 'Summer 2025 (via Alexey).', 'section...",NaN
1,documents_20240821_2955,lqBXcpEBGUgXSWImI71R,5.754293,"{'text': 'No, you can only get a certificate i...",[text.keyword]
2,documents_20240821_2955,WKBXcpEBGUgXSWImG72P,4.399607,{'text': 'This is likely to be an error when i...,NaN
3,documents_20240821_2955,e6BXcpEBGUgXSWImH73N,4.220145,{'text': 'Cosine similarity is a measure used ...,[text.keyword]
4,documents_20240821_2955,lKBXcpEBGUgXSWImI70M,4.070828,{'text': 'The error indicates that you have no...,NaN


Q6. Reindexing

In [23]:
index_name2 = 'documents_20240821_3753'
response = es_client.search(index=index_name2, body=search_query)
df2 = pd.DataFrame(response['hits']['hits'])
df2


,_index,_id,_score,_source,_ignored
0,documents_20240821_3753,DaBecpEBGUgXSWImZL6A,8.443945,"{'text': 'Summer 2025 (via Alexey).', 'section...",NaN
1,documents_20240821_3753,QqBecpEBGUgXSWImaL7a,5.754293,"{'text': 'No, you can only get a certificate i...",[text.keyword]
2,documents_20240821_3753,BKBecpEBGUgXSWImY76W,4.399607,{'text': 'This is likely to be an error when i...,NaN
3,documents_20240821_3753,J6BecpEBGUgXSWImZr7u,4.220145,{'text': 'Cosine similarity is a measure used ...,[text.keyword]
4,documents_20240821_3753,QKBecpEBGUgXSWImaL7B,4.070828,{'text': 'The error indicates that you have no...,NaN
